# Import Libraries

In [1]:
import utils
from utils import *

Using TensorFlow backend.


In [3]:
#parameters
current_dir = os.getcwd();
custom_bucket_boundary=[0,1,1000,200000] 
vocab_size=10000
max_len=200
max_words=10000
embed_size=300
max_features=10000
#sample=150 #sample reocrds
#train_size=1000#sample train
dropout_rate=0.2
size_embedding=10000
epochs=50
num_gpu=2
#tensorboard_dir="./Payout" #tensor board sub directory
lbl = LabelEncoder()

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## GPU and Tensorflow check

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17378274702901112497
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11280557671
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 16854966812959137785
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:0e:00.0, compute capability: 3.7"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 11280557671
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 408489061951198463
physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0000:0f:00.0, compute capability: 3.7"
]


In [5]:
import tensorflow as tf

In [10]:
tf.__version__

'1.11.0'

In [12]:
#Tensorflow check
tf.Session()

with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3]) 
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

print(sess.run(c))

# Runs the op.
# Log information
options = tf.RunOptions(output_partition_graphs=True)
metadata = tf.RunMetadata()
c_val = sess.run(c, options=options, run_metadata=metadata)

print(metadata.partition_graphs)

sess.close()

[[22. 28.]
 [49. 64.]]
[node {
  name: "MatMul/_0"
  op: "_Send"
  input: "MatMul/_0__cf__0"
  device: "/job:localhost/replica:0/task:0/device:GPU:0"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "client_terminated"
    value {
      b: false
    }
  }
  attr {
    key: "recv_device"
    value {
      s: "/job:localhost/replica:0/task:0/device:CPU:0"
    }
  }
  attr {
    key: "send_device"
    value {
      s: "/job:localhost/replica:0/task:0/device:GPU:0"
    }
  }
  attr {
    key: "send_device_incarnation"
    value {
      i: 1
    }
  }
  attr {
    key: "tensor_name"
    value {
      s: "edge_7_MatMul"
    }
  }
}
node {
  name: "MatMul/_0__cf__0"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:GPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
          

# Data Import

In [4]:
import os
current_dir = os.getcwd()

In [5]:
current_dir

'/notebooks'

In [7]:
%mkdir model#model directory

mkdir: cannot create directory 'model#model': File exists
mkdir: cannot create directory 'directory': File exists


In [6]:
model_save_dir = os.getcwd() + '/model/target_encoded_conv_69_features.hdf5' # save best models 

In [128]:
%mkdir tensorBo #tensorboard directory
tensorboard_dir=os.getcwd() + "/tensorBo/" #tensor board graphs

In [7]:
path = current_dir+'/totalloss_preprocess/'

In [18]:
df = pd.read_csv(path+'final_train_with_text_feature_encoded.csv')

FileNotFoundError: File b'/notebooks/totalloss_preprocess/scrubbed-final_train_with_text_feature_encoded.csv' does not exist

In [11]:
df.shape

(1261439, 72)

In [13]:
train_claim_numbers = df.claim_claimnumber.copy()
df.drop('claim_claimnumber',axis=1,inplace=True)
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [14]:
test = pd.read_csv(path+'test_full_clean_data_with_encoding.csv')

In [15]:
test.shape

(100574, 72)

In [17]:
test.exp_claimorder

AttributeError: 'DataFrame' object has no attribute 'exp_claimorder'

In [12]:
test_claims_numbers=test.claim_claimnumber

In [13]:
test_exp_claimorder=test.exp_claimorder

In [14]:
test.drop(['claim_claimnumber','exp_claimorder'],axis=1,inplace=True)

In [15]:
test.drop('Unnamed: 0',axis=1,inplace=True)

In [58]:
(df.shape,test.shape)

((1261439, 73), (76385, 73))

In [33]:
for col in df.columns:
    if col not in test.columns:
        print('{}'.format(col))

full_text


In [34]:
df.drop('full_text',axis=1,inplace=True)

In [12]:
for col in df.columns:
    if df[col].dtype=='O':
        print('{}'.format(col))
        df.drop(col,axis=1,inplace=True)

claim_description
inc_description
inc_vehdamagesnapshot_aaa
vehicle_properties
text_combines
claim_claimnumber


In [31]:
df.claim_claimnumber

AttributeError: 'DataFrame' object has no attribute 'claim_claimnumber'

In [25]:
train_totalloss=df.inc_totalloss.copy()

In [49]:
start = time.time()
dct.fit(df,totalloss)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [50]:
print('{} mins'.format((time.time()-start)/60))

1.4122042655944824 mins


# Text Features handling

In [59]:
numerical_cols = ['vehicle_year','vehicle_mileage','points','mileage','coveage_deductible','vehicle_age',
                 'word_count','unique_word_count','char_count','word_density']
text_cols = ['claim_loss_description','incident_loss_description','','vehicle_properties','text_combines']
cat_cols = [col for col in df.columns if col not in numerical_cols + text_cols]

In [35]:
text_cols=[]
for col in df.columns:
    if df[col].dtype=='O':
        print('{}'.format(col))
        text_cols.append(col)


claim_description
inc_description
inc_vehdamagesnapshot_aaa
vehicle_properties
text_combines


In [37]:
#parameters
current_dir = os.getcwd();
custom_bucket_boundary=[0,1,1000,200000] 
vocab_size=10000
max_len=500
max_words=10000
embed_size=300
max_features=10000
sample=150 #sample reocrds
train_size=1000#sample train
dropout_rate=0.2
size_embedding=10000
epochs=50
num_gpu=3
#tensorboard_dir="./Payout" #tensor board sub directory
lbl = LabelEncoder()

In [38]:
df_combine=pd.concat([df,test])

In [60]:
len(df)

1261439

In [22]:
#df_text['full_text'] = ''
#for f in text_cols:
 #   if text_cols!=
df['full_text'] = df['text_combines'] + df['vehicle_properties']

In [39]:
df_combine['full_text']= df_combine['text_combines'] + df_combine['vehicle_properties']

In [62]:
tknizer = Tokenizer(num_words=max_words)
tknizer.fit_on_texts(df_combine['full_text']) 

In [63]:
train_text = tknizer.texts_to_sequences(df_combine['full_text'][:1261439]) #text feature
test_text=tknizer.texts_to_sequences(df_combine['full_text'][1261439:])

In [64]:
train_text = pad_sequences(train_text,maxlen=max_len)

In [65]:
test_text=pad_sequences(test_text,maxlen=max_len)

In [57]:
train_text.shape

AttributeError: 'list' object has no attribute 'shape'

In [56]:
test_text.shape

AttributeError: 'list' object has no attribute 'shape'

In [56]:
pd.options.display.float_format = '{:,.2f}'.format

In [57]:
df.word_count.describe()

count   1,261,439.00
mean            0.08
std             0.05
min             0.00
25%             0.03
50%             0.07
75%             0.10
max             1.00
Name: word_count, dtype: float64

In [66]:
embed_size =300
max_features = 10000
max_len = 200

In [67]:
EMBEDDING_FILE= "glove.840B.300d.txt"

In [68]:
def get_coeff(word,*arr): return word, np.asarray(arr,dtype='float32')
embedding_index = dict(get_coeff(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE,encoding='utf-8'))

In [69]:
word_index = tknizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words,embed_size)) #np.random.normal(emb_mean,emb_std,(nb_words,embed_size))
for word,i in word_index.items():
    if  i >= max_features : continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None : embedding_matrix[i] = embedding_vector

In [70]:
embedding_matrix.shape

(10000, 300)

# Target encoding

In [ ]:
#drop train text columns
for col in df.columns:
    if df[col].dtype=='O':
        print('{}'.format(col))
        df.drop(col,axis=1,inplace=True)

In [ ]:
#drop test text columns
for col in test.columns:
    if test[col].dtype=='O':
        print('{}'.format(col))
        test.drop(col,axis=1,inplace=True)

In [116]:
train_cat=df[cat_cols]
print('Shape of train categorical features {}'.format(train_cat.shape))
test_cat=test[cat_cols]
print('Shape of test categorical features{}'.format(test_cat.shape))

Shape of train numerical features (1261439, 57)
Shape of test numerical features(76385, 57)


In [118]:
print('Shape of train text features {}'.format(train_text.shape))
print('Shape of test text features{}'.format(test_text.shape))

Shape of train text features (1261439, 500)
Shape of test text features(76385, 500)


In [115]:
train_num=df[numerical_cols]
print('Shape of train numerical features {}'.format(train_num.shape))
test_num=test[numerical_cols]
print('Shape of test numerical features{}'.format(test_num.shape))

Shape of train numerical features (1261439, 10)
Shape of test numerical features(76385, 10)


In [95]:
test_loss=test.inc_totalloss

In [94]:
train_totalloss=df.inc_totalloss.copy()

In [109]:
train_totalloss.replace(19,1,inplace=True)

In [101]:
test.drop('inc_totalloss',axis=1,inplace=True)

In [103]:
df.drop('inc_totalloss',axis=1,inplace=True)

In [105]:
test.shape

(76385, 67)

In [110]:
#target variable one hot 
train_target= tf.keras.utils.to_categorical(train_totalloss,num_classes=None)
print("Output shape {}".format(train_target.shape))

Output shape (1261439, 2)


In [107]:
#target variable one hot 
test_target= tf.keras.utils.to_categorical(test_loss,num_classes=None)
print("Output shape {}".format(test_target.shape))

Output shape (76385, 2)


In [ ]:
for col in test.columns:
    if test[col].dtype=='O':
        print('{}'.format(col))
        test.drop(col,axis=1,inplace=True)

# Models

In [134]:
from keras.layers import GlobalAveragePooling1D,GlobalMaxPooling1D
def get_model():    
        # categorical channel
        with tf.name_scope("Input_Category"):      
            inputs1 = Input(shape=(train_cat.shape[1],))
        with tf.name_scope("Dense_cat"): 
            dense_cat_1 = Dense(256, activation='relu')(inputs1)
            dense_cat_2 = Dense(128, activation='relu')(dense_cat_1)
        with tf.name_scope("Dropout"):
            drop1 = Dropout(0.2)(dense_cat_2)
        with tf.name_scope('Flat_1'):
            flat1 = Dense(32, activation='relu')(drop1)
            
    #numeric channel
        with tf.name_scope("Numeric_channel"):
            inputs2 = Input(shape=(train_num.shape[1],))
        with tf.name_scope("Dense_real"):
            dense_real_1=Dense(256,activation='relu')(inputs2)
            dense_real_2 = Dense(128,activation='relu')(dense_real_1) 
        with tf.name_scope("Dropout"):
            drop2 = Dropout(0.2)(dense_real_2)
        with tf.name_scope('Flat_2'):
            flat2=Dense(64,activation='relu')(drop2)

        # text chanel
        with tf.name_scope("Input_Text"):
            inputs3 = Input(shape=(train_text.shape[1],))
            embedding3 = Embedding(size_embedding, 300,weights=[embedding_matrix],trainable = False)(inputs3)
            x = SpatialDropout1D(0.2)(embedding3)
        #with tf.name_scope("LSTM"):
            #x1 = Bidirectional(GRU(80,return_sequences=True,dropout=0.2,recurrent_dropout=0.25))(x)
        with tf.name_scope("Convolution"):
            conv3 = Conv1D(filters=128, kernel_size=8,padding='valid',kernel_initializer='glorot_uniform')(x)
        with tf.name_scope("Dropout"):
            drop3 = Dropout(0.2)(conv3)
        with tf.name_scope('Average_pool'):
            avg_pool = GlobalAveragePooling1D()(drop3)
        with tf.name_scope('MaxPool'):
            max_pool = GlobalMaxPooling1D()(drop3)
        with tf.name_scope("Concat"):
            x = concatenate([avg_pool,max_pool])
       # with tf.name_scope('Flat_3'):
          #  flat3 = Flatten()(x)

        # merge
        with tf.name_scope('Merge_Channels'):
            merged = concatenate([flat1,x])
        with tf.name_scope("Dense"):
            dense1 = Dense(200, activation='relu')(merged)
            dense2 = Dense(100, activation='relu')(dense1)
        with tf.name_scope("Output"):
            outputs = Dense(2, activation='sigmoid')(merged)
        model1 = Model(inputs=[inputs1,inputs2,inputs3], outputs=outputs)
        return model1

In [135]:
model = get_model()
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 500, 300)     3000000     input_9[0][0]                    
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 57)           0                                            
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, 500, 300)     0           embedding_3[0][0]                
__________________________________________________________________________________________________
dense_16 (

# Compile & Training

In [1]:
from keras.optimizers import SGD,Adamax,Nadam

Using TensorFlow backend.


In [130]:
from keras.callbacks import ReduceLROnPlateau

In [2]:
df_combine

NameError: name 'df_combine' is not defined

In [131]:
#CallBacks
#check_point = ModelCheckpoint(model_save_dir, monitor = "val_loss", verbose = 1,
                              save_best_only = True, mode = "min")
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)
#tbCallBack = TensorBoard(log_dir=tensorboard_dir,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)

In [136]:
with tf.device('/device:cpu:0'):
    with tf.name_scope('Compile'):
        model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=1e-5),metrics=['accuracy'])

## Training

In [1]:
#for d in ['/device:GPU:0','/device:GPU:1','/device:GPU:2']:
with tf.device('/device:GPU:0'):
    model.fit([train_cat,train_num, train_text], train_target, batch_size=128, epochs=20, validation_split=0.15,callbacks=[check_point,early_stop,tbCallBack,learning_rate_reduction])

NameError: name 'tf' is not defined

In [2]:
import gc
gc.collect()

0

## XGBoost with cross val

In [24]:
#xgboost params
from sklearn import feature_selection
import xgboost as xgb
#import lightgbm as lgb

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn import preprocessing
from scipy.sparse import hstack, csr_matrix
import scipy.stats as st
from sklearn.model_selection import StratifiedKFold

In [33]:
test.drop('exp_claimorder',axis=1,inplace=True)

In [32]:
for col in test.columns:
    if col not in df.columns:
        print('{}'.format(col))

exp_claimorder


In [34]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return [('gini', gini_score)]

def gini_lgb(preds, dtrain):
    y = list(dtrain.get_label())
    score = gini(y, preds) / gini(y, y)
    return 'gini', score, True

In [47]:
train=df.copy()

In [53]:
train.shape

(1261439, 67)

In [ ]:
nrounds=1000  # need to change to 2000
kfold = 5  # need to change to 5
skf = StratifiedKFold(n_splits=kfold, random_state=0)
for i, (train_index, test_index) in enumerate(skf.split(train, totalloss)):
    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_valid = train.iloc[train_index], train.iloc[test_index]
    y_train, y_valid = totalloss.iloc[train_index], totalloss.iloc[test_index]
    d_train = xgb.DMatrix(X_train, y_train) 
    d_valid = xgb.DMatrix(X_valid, y_valid) 
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    xgb_model = xgb.train(xgb_params, d_train, nrounds, watchlist, early_stopping_rounds=100, 
                          feval=gini_xgb, maximize=True, verbose_eval=100)
    sub['target'] += xgb_model.predict(xgb.DMatrix(test), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (kfold)
sub.to_csv('xgboost_5fold.csv', index=False, float_format='%.5f') 
gc.collect()
sub.head(2)

 xgb kfold: 1  of  5 : 
[18:36:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1900 extra nodes, 0 pruned nodes, max_depth=12
[0]	train-logloss:0.647797	valid-logloss:0.647483	train-gini:0.993125	valid-gini:0.994145
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[18:36:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2756 extra nodes, 0 pruned nodes, max_depth=12
[18:36:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2008 extra nodes, 0 pruned nodes, max_depth=12
[18:36:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2830 extra nodes, 0 pruned nodes, max_depth=12
[18:36:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2006 extra nodes, 0 pruned nodes, max_depth=12
[18:36:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1872 extra nodes, 0 pruned node

[18:37:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2052 extra nodes, 0 pruned nodes, max_depth=12
[18:37:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2090 extra nodes, 0 pruned nodes, max_depth=12
[18:38:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2162 extra nodes, 0 pruned nodes, max_depth=12
[18:38:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2082 extra nodes, 0 pruned nodes, max_depth=12
[18:38:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2042 extra nodes, 0 pruned nodes, max_depth=12
[18:38:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2384 extra nodes, 0 pruned nodes, max_depth=12
[18:38:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2344 extra nodes, 0 pruned nodes, max_depth=12
[18:38:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2214 extra nodes, 0 pruned nodes, max_d

[18:39:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1334 extra nodes, 0 pruned nodes, max_depth=12
[18:39:12] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1440 extra nodes, 0 pruned nodes, max_depth=12
[18:39:13] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1154 extra nodes, 0 pruned nodes, max_depth=12
[18:39:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 978 extra nodes, 0 pruned nodes, max_depth=12
[18:39:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1470 extra nodes, 0 pruned nodes, max_depth=12
[18:39:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 980 extra nodes, 0 pruned nodes, max_depth=12
[18:39:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1086 extra nodes, 0 pruned nodes, max_depth=12
[18:39:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1258 extra nodes, 0 pruned nodes, max_dep

[18:40:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 322 extra nodes, 0 pruned nodes, max_depth=12
[18:40:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 928 extra nodes, 0 pruned nodes, max_depth=12
[18:40:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 302 extra nodes, 0 pruned nodes, max_depth=12
[18:40:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 382 extra nodes, 0 pruned nodes, max_depth=12
[18:40:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1720 extra nodes, 0 pruned nodes, max_depth=12
[18:40:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1216 extra nodes, 0 pruned nodes, max_depth=12
[18:40:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 976 extra nodes, 0 pruned nodes, max_depth=12
[200]	train-logloss:0.023153	valid-logloss:0.03286	train-gini:0.999747	valid-gini:0.997726
[18:40:36] /workspace/src/tree/u

[18:41:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 598 extra nodes, 0 pruned nodes, max_depth=12
[18:41:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 992 extra nodes, 0 pruned nodes, max_depth=12
[18:41:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1194 extra nodes, 0 pruned nodes, max_depth=12
[18:41:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 700 extra nodes, 0 pruned nodes, max_depth=12
[18:41:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1454 extra nodes, 0 pruned nodes, max_depth=12
[18:41:50] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 834 extra nodes, 0 pruned nodes, max_depth=12
[18:41:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 862 extra nodes, 0 pruned nodes, max_depth=12
[18:41:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1890 extra nodes, 0 pruned nodes, max_depth=

[18:43:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 728 extra nodes, 0 pruned nodes, max_depth=12
[18:43:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1152 extra nodes, 0 pruned nodes, max_depth=12
[18:43:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1394 extra nodes, 0 pruned nodes, max_depth=12
[18:43:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1564 extra nodes, 0 pruned nodes, max_depth=12
[18:43:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1676 extra nodes, 0 pruned nodes, max_depth=12
[18:43:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1432 extra nodes, 0 pruned nodes, max_depth=12
[18:43:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1792 extra nodes, 0 pruned nodes, max_depth=12
[18:43:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1776 extra nodes, 0 pruned nodes, max_de

[18:44:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1300 extra nodes, 0 pruned nodes, max_depth=12
[18:44:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1522 extra nodes, 0 pruned nodes, max_depth=12
[18:44:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1772 extra nodes, 0 pruned nodes, max_depth=12
[18:44:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1414 extra nodes, 0 pruned nodes, max_depth=12
[18:44:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1450 extra nodes, 0 pruned nodes, max_depth=12
[18:44:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1138 extra nodes, 0 pruned nodes, max_depth=12
[18:44:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 960 extra nodes, 0 pruned nodes, max_depth=12
[18:44:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1684 extra nodes, 0 pruned nodes, max_de

[18:45:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1450 extra nodes, 0 pruned nodes, max_depth=12
[18:45:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1568 extra nodes, 0 pruned nodes, max_depth=12
[18:45:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1270 extra nodes, 0 pruned nodes, max_depth=12
[18:45:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1130 extra nodes, 0 pruned nodes, max_depth=12
[18:45:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1458 extra nodes, 0 pruned nodes, max_depth=12
[18:45:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1258 extra nodes, 0 pruned nodes, max_depth=12


In [50]:
xgb_params = {'eta': 0.05, 
              'max_depth': 12, 
              'subsample': 0.8, 
              'colsample_bytree': 0.75,
              #'min_child_weight' : 1.5,
              'scale_pos_weight': 8.7,
              'objective': 'binary:logistic', 
              'eval_metric': 'logloss', 
              'seed': 23,
              'lambda': 1.5,
              'alpha': .6
             }

In [40]:
sub=pd.DataFrame(test_claims_numbers,columns=['claim_claimnumber'])

In [42]:
sub['exp_claimorder']=test_exp_claimorder

In [45]:
sub.head()

,claim_claimnumber,exp_claimorder,actual_loss
0,1002-65-5100,1,0
1,1002-65-5104,1,0
2,1002-65-5118,1,0
3,1002-65-5120,1,0
4,1002-65-5121,1,1


In [44]:
sub['actual_loss']=actual_loss

In [54]:
sub['target']=0

In [56]:
totalloss.value_counts()

0     1131424
1      130014
19          1
Name: inc_totalloss, dtype: int64

In [62]:
totalloss.loc[totalloss==19,'totalloss']=1

IndexingError: (0          False
1          False
2          False
3          False
4          False
5          False
6          False
7          False
8          False
9          False
10         False
11         False
12         False
13         False
14         False
15         False
16         False
17         False
18         False
19         False
20         False
21         False
22         False
23         False
24         False
25         False
26         False
27         False
28         False
29         False
           ...  
1261409    False
1261410    False
1261411    False
1261412    False
1261413    False
1261414    False
1261415    False
1261416    False
1261417    False
1261418    False
1261419    False
1261420    False
1261421    False
1261422    False
1261423    False
1261424    False
1261425    False
1261426    False
1261427    False
1261428    False
1261429    False
1261430    False
1261431    False
1261432    False
1261433    False
1261434    False
1261435    Fals

In [63]:
totalloss.replace(19,1,inplace=True)

In [64]:
totalloss.value_counts()

0    1131424
1     130015
Name: inc_totalloss, dtype: int64